In [3]:
import numpy as np
import pandas as pd
import os

from PIL import Image
from tflearn.data_utils import build_hdf5_image_dataset

TRAIN = 'training.csv'
#VAL = 'validation.csv'
TRAIN_TRIM = 'training_trim.csv'

#FULL_COLS = ['file_path','face_x','face_y','face_width','face_height','facial_landmarks','expression','valence','arousal']
#TRIM_COLS = ['file_path', 'face_x', 'face_y', 'face_width', 'face_height', 'expression']

#training = pd.read_csv(os.environ['HOME'] + '/' + TRAIN)
#validation = pd.read_csv(VAL, names=FULL_COLS)

#training.columns = FULL_COLS
#training = training[TRIM_COLS]
#training.to_csv(TRAIN_TRIM)

#TRAIN_FILE = 'training_trim.csv'

train = pd.read_csv(TRAIN_TRIM)
#DATA_DIR = '/home/maxschorer/affectnet_data'
# split into training and test

In [4]:
#train.head()
DATA_DIR = '/home/maxschorer/test'

folders = train['file_path'].map(lambda x: x.split('/')[0])
for folder in folders:
    new_folder = DATA_DIR + '/' + folder
    if os.path.exists(new_folder): continue
    os.makedirs(new_folder)

In [5]:
train.head()

,Unnamed: 0,file_path,face_x,face_y,face_width,face_height,expression
0,0,689/737db2483489148d783ef278f43f486c0a97e140fc...,134.0,134.0,899,899,1
1,1,392/c4db2f9b7e4b422d14b6e038f0cdc3ecee239b5532...,20.0,20.0,137,137,0
2,2,468/21772b68dc8c2a11678c8739eca33adb6ccc658600...,11.0,11.0,176,176,0
3,3,944/06e9ae8d3b240eb68fa60534783eacafce2def60a8...,40.0,40.0,269,269,1
4,4,993/02e06ee5521958b4042dd73abb444220609d96f57b...,22.0,22.0,153,153,8


In [6]:
#create folders, move to 
DATA_DIR = '/home/maxschorer/test'
#val = pd.read_csv('validation.csv')

for idx, row in train.iterrows():
    old_file_path = DATA_DIR + '/' + row['file_path'].split('/')[1]
    if not os.path.isfile(old_file_path): continue
    new_file_path = DATA_DIR + '/' + row['file_path']
    os.rename(old_file_path, new_file_path)

In [11]:
val_indexes = np.random.choice(train.shape[0], int(train.shape[0]/10), replace=False)

In [22]:
TRAIN_DATASET_FILE = 'train_dataset.txt'
VAL_DATASET_FILE = 'val_dataset.txt'

train_f = open(TRAIN_DATASET_FILE, 'w')
val_f = open(VAL_DATASET_FILE, 'w')

for idx, row in train.iterrows():
    file_path = DATA_DIR + '/' + row['file_path']
    if not os.path.isfile(file_path): continue
    try:
        Image.open(file_path)
    except Exception as e:
        print('Unable to open {}'.format(file_path))
        continue
    line_string = '{} {}\n'.format(file_path, row['expression'])
    if idx in val_indexes:
        val_f.write(line_string)
    else:
        train_f.write(line_string)
    # crop image
    # write logic to toss out poorly formatted images

train_f.close()
val_f.close()

Unable to open /home/maxschorer/affectnet_data/103/29a31ebf1567693f4644c8ba3476ca9a72ee07fe67a5860d98707a0a.jpg


In [2]:
TRAIN_DATASET_FILE = 'train_dataset.txt'
TRAIN_H5 = 'train.h5'
IMG_SIZE = 256
%time build_hdf5_image_dataset(TRAIN_DATASET_FILE, image_shape=(IMG_SIZE, IMG_SIZE), mode='file', output_path=TRAIN_H5, normalize=True, categorical_labels=True)

CPU times: user 21min 19s, sys: 1min 32s, total: 22min 51s
Wall time: 1h 16min 22s


In [23]:
VAL_H5 = 'val.h5'
VAL_DATASET_FILE = 'val_dataset.txt'

%time build_hdf5_image_dataset(VAL_DATASET_FILE, image_shape=(IMG_SIZE, IMG_SIZE), mode='file', output_path=VAL_H5, normalize=True, categorical_labels=True)

CPU times: user 2min 20s, sys: 9.33 s, total: 2min 30s
Wall time: 5min 47s
